In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
import os
from tqdm import tqdm as tqdm
import datetime

In [2]:
page_path_whole = '/home3/r05322021/Desktop/FB Data/Polarization/page_score/whole/'
page_score_file_whole = os.listdir(page_path_whole)
page_path_all = '/home3/r05322021/Desktop/FB Data/original_ideology/page_score/2015-05-03_to_2016-11-26_4weeks/'
page_score_file_all = os.listdir(page_path_all)
page_path_new = '/home3/r05322021/Desktop/FB Data/Polarization/page_score/new/all/'
page_score_file_new = os.listdir(page_path_new)
page_path_quit = '/home3/r05322021/Desktop/FB Data/Polarization/page_score/quit/all/'
page_score_file_quit = os.listdir(page_path_quit)

In [3]:
df_whole_list, df_all_list, df_new_list, df_quit_list = [], [], [], []
for f in page_score_file_whole:
    df = pd.read_csv(f'{page_path_whole}{f}', converters={'page_id': str}, engine='python')
    df['Week'] = f[-14:-4]
    df_whole_list.append(df)
for f in page_score_file_all:
    df = pd.read_csv(f'{page_path_all}{f}', converters={'page_id': str}, engine='python')
    df['Week'] = f[-14:-4]
    df_all_list.append(df)
for f in page_score_file_new:
    df = pd.read_csv(f'{page_path_new}{f}', converters={'page_id': str}, engine='python')
    df['Week'] = f[-14:-4]
    df_new_list.append(df)
for f in page_score_file_quit:
    df = pd.read_csv(f'{page_path_quit}{f}', converters={'page_id': str}, engine='python')
    df['Week'] = f[-14:-4]
    df_quit_list.append(df)
df_whole = pd.concat(df_whole_list, axis=0)
df_all = pd.concat(df_all_list, axis=0)
df_new = pd.concat(df_new_list, axis=0)
df_quit = pd.concat(df_quit_list, axis=0)

In [4]:
std_whole = pd.DataFrame([[d['Week'].iloc[0], d['PC1_std'].std()] for d in [group[1] for group in df_whole.groupby('Week')]], columns=['Week', 'std_whole'])
std_all = pd.DataFrame([[d['Week'].iloc[0], d['PC1_std'].std()] for d in [group[1] for group in df_all.groupby('Week')]], columns=['Week', 'std_all'])
std_new = pd.DataFrame([[d['Week'].iloc[0], d['PC1_std'].std()] for d in [group[1] for group in df_new.groupby('Week')]], columns=['Week', 'std_new'])
std_quit = pd.DataFrame([[d['Week'].iloc[0], d['PC1_std'].std()] for d in [group[1] for group in df_quit.groupby('Week')]], columns=['Week', 'std_quit'])

In [39]:
std_whole['Week'] = std_whole['Week'].apply(lambda x: str(datetime.datetime.strptime(x, '%Y-%m-%d').date() - datetime.timedelta(days=6)))
std_all['Week'] = std_all['Week'].apply(lambda x: str(datetime.datetime.strptime(x, '%Y-%m-%d').date() - datetime.timedelta(days=6)))
std_new['Week'] = std_new['Week'].apply(lambda x: str(datetime.datetime.strptime(x, '%Y-%m-%d').date() - datetime.timedelta(days=6)))
std_quit['Week'] = std_quit['Week'].apply(lambda x: str(datetime.datetime.strptime(x, '%Y-%m-%d').date() - datetime.timedelta(days=6)))

In [13]:
reaction = pd.read_csv('/home3/r05322021/Desktop/FB Data/issue_reaction/whole/reaction_weekly.csv')
user = pd.read_csv('/home3/r05322021/Desktop/FB Data/issue_reaction/whole/user_weekly.csv')
post = pd.read_csv('/home3/r05322021/Desktop/FB Data/issue_reaction/whole/post_weekly.csv')
reaction = reaction[reaction.Week < '2016-11-27']
user = user[user.Week < '2016-11-27']
post = post[post.Week < '2016-11-27']

In [15]:
reaction_immigration = reaction[reaction.type == 'immigration'][['Week', 'amount', 'ratio']].rename(columns={'amount': 'immigration_reaction_amount', 'ratio': 'immigration_reaction_ratio'})
reaction_race = reaction[reaction.type == 'race'][['Week', 'amount', 'ratio']].rename(columns={'amount': 'race_reaction_amount', 'ratio': 'race_reaction_ratio'})
user_immigration = user[user.type == 'immigration'][['Week', 'amount', 'ratio']].rename(columns={'amount': 'immigration_user_amount', 'ratio': 'immigration_user_ratio'})
user_race = user[user.type == 'race'][['Week', 'amount', 'ratio']].rename(columns={'amount': 'race_user_amount', 'ratio': 'race_user_ratio'})
post_immigration = post[post.type == 'immigration'][['Week', 'amount', 'ratio']].rename(columns={'amount': 'immigration_post_amount', 'ratio': 'immigration_post_ratio'})
post_race = post[post.type == 'race'][['Week', 'amount', 'ratio']].rename(columns={'amount': 'race_post_amount', 'ratio': 'race_post_ratio'})

In [27]:
FB_df = pd.merge(reaction_immigration, reaction_race, how='inner', on=['Week'])
for d in [user_immigration, user_race]:
    FB_df = pd.merge(FB_df, d, how='inner', on=['Week'])
for d in [post_immigration, post_race]:
    FB_df = pd.merge(FB_df, d, how='inner', on=['Week'])
for col in ['reaction_amount', 'reaction_ratio', 'user_amount', 'user_ratio', 'post_amount', 'post_ratio']:
    FB_df[f'related_{col}'] = FB_df[f'immigration_{col}'] + FB_df[f'race_{col}']

In [42]:
for df in [std_whole, std_all, std_new, std_quit]:
    FB_df = pd.merge(FB_df, df, how='inner', on='Week')

In [43]:
FB_df.to_csv()

,Week,immigration_reaction_amount,immigration_reaction_ratio,race_reaction_amount,race_reaction_ratio,immigration_user_amount,immigration_user_ratio,race_user_amount,race_user_ratio,immigration_post_amount,...,related_reaction_ratio,related_user_amount,related_user_ratio,related_post_amount,related_post_ratio,std_whole_x,std_whole_y,std_all,std_new,std_quit
0,2016-07-24,2885263.0,0.026049,2825095.0,0.025506,1595933.0,0.128812,1526362.0,0.123197,2934,...,0.051555,3122295.0,0.252009,7079,0.043007,1.013407,1.013407,1.021156,1.015074,1.017447
1,2015-10-25,2023930.0,0.021353,1911064.0,0.020162,839970.0,0.076317,941576.0,0.085548,3942,...,0.041516,1781546.0,0.161865,7226,0.047026,1.004979,1.004979,1.006763,1.005281,1.006037
2,2016-07-17,1425550.0,0.014049,4071174.0,0.040122,904224.0,0.076219,1946117.0,0.164042,2707,...,0.054171,2850341.0,0.240261,9149,0.054621,1.014631,1.014631,1.021681,1.016420,1.018159
3,2015-06-28,3710509.0,0.037015,2512224.0,0.025062,1696206.0,0.157087,1262465.0,0.116918,3391,...,0.062077,2958671.0,0.274005,6763,0.052692,1.003497,1.003497,1.004652,1.003558,1.004238
4,2016-04-17,1379229.0,0.016774,983745.0,0.011964,839057.0,0.071241,640050.0,0.054344,2640,...,0.028739,1479107.0,0.125586,4843,0.031482,1.011890,1.011890,1.018039,1.012961,1.014888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,2016-10-02,1247237.0,0.012671,2038835.0,0.020712,749309.0,0.062031,1224392.0,0.101361,2506,...,0.033383,1973701.0,0.163392,5827,0.034684,1.018293,1.018293,1.027001,1.020533,1.022483
75,2015-11-08,3033042.0,0.028934,1590739.0,0.015175,1254223.0,0.103582,850146.0,0.070210,4163,...,0.044109,2104369.0,0.173792,7284,0.046521,1.005954,1.005954,1.007097,1.006413,1.006265
76,2016-10-23,781820.0,0.007961,1707909.0,0.017391,518462.0,0.044546,1012311.0,0.086977,2079,...,0.025353,1530773.0,0.131523,5111,0.030274,1.018476,1.018476,1.026311,1.019622,1.021446
77,2016-07-03,1370080.0,0.014621,4658694.0,0.049716,759735.0,0.061652,2206981.0,0.179096,2552,...,0.064337,2966716.0,0.240749,9318,0.062257,1.015043,1.015043,1.022445,1.017279,1.018079


In [31]:
FB_df.Week.unique()

array(['2016-07-24', '2015-10-25', '2016-07-17', '2015-06-28',
       '2016-04-17', '2015-12-20', '2015-05-31', '2015-06-07',
       '2016-02-21', '2015-08-30', '2016-03-27', '2015-08-23',
       '2016-06-12', '2015-11-29', '2015-08-16', '2016-11-06',
       '2016-03-13', '2015-09-06', '2016-07-31', '2016-02-14',
       '2015-08-02', '2015-05-17', '2015-11-01', '2016-09-25',
       '2016-10-09', '2015-05-24', '2016-11-13', '2016-05-01',
       '2016-08-21', '2016-04-24', '2016-01-10', '2015-10-11',
       '2016-03-06', '2015-11-15', '2016-06-05', '2016-01-24',
       '2015-12-27', '2015-07-12', '2015-07-26', '2016-05-08',
       '2016-04-03', '2016-02-07', '2015-12-13', '2016-08-28',
       '2016-09-18', '2016-06-26', '2015-06-14', '2016-08-14',
       '2016-02-28', '2016-11-20', '2016-09-11', '2016-01-03',
       '2016-01-17', '2016-09-04', '2015-09-13', '2016-06-19',
       '2015-07-05', '2015-09-27', '2016-08-07', '2015-06-21',
       '2016-03-20', '2015-12-06', '2015-10-18', '2016-

In [38]:
str(datetime.datetime.strptime('2015-05-30', '%Y-%m-%d').date() - datetime.timedelta(days=6))

'2015-05-24'